In [13]:
using Gumbo
using Cascadia
using DataFrames

ArgumentError: ArgumentError: Package DataFrames not found in current path:
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.


In [ ]:

url = "https://www.velogames.com/velogame/2022/riders.php"

page = parsehtml(read(download(url), String))


In [12]:
rider_table = eachmatch(sel"table", page.root)[1]
rider_names = String[]
rider_classes = String[]
rider_costs = Int64[]
rider_scores = Int64[]
for rider_row in eachmatch(sel"tr", rider_table)[2:end]
    #@show rider_row
    rider_cells = eachmatch(sel"td", rider_row)
    push!(rider_names, text(rider_cells[2]))
    push!(rider_classes, text(rider_cells[4]))
    push!(rider_costs, parse(Int64, text(rider_cells[5])))
    push!(rider_scores, parse(Int64, text(rider_cells[7])))
end
@show rider_scores

rider_scores = [2534, 677, 2424, 2162, 1050, 215, 737, 530, 1378, 607, 8, 865, 185, 560, 552, 0, 1012, 515, 194, 794, 532, 760, 767, 538, 183, 48, 155, 422, 91, 92, 706, 12, 197, 581, 932, 338, 236, 282, 512, 23, 20, 154, 411, 128, 954, 114, 367, 0, 52, 42, 543, 53, 62, 20, 0, 0, 96, 65, 390, 28, 349, 431, 43, 435, 47, 276, 130, 118, 423, 492, 0, 0, 80, 442, 52, 743, 34, 296, 438, 179, 374, 342, 48, 646, 195, 256, 138, 38, 296, 545, 324, 332, 860, 56, 3, 225, 10, 44, 485, 48, 252, 41, 113, 159, 407, 59, 335, 155, 353, 18, 54, 42, 38, 98, 305, 8, 48, 9, 8, 288, 94, 184, 4, 12, 297, 162, 222, 34, 230, 56, 115, 153, 250, 32, 28, 18, 6, 61, 23, 20, 0, 20, 0, 52, 50, 133, 0, 36, 26, 101, 277, 24, 45, 34, 54, 166, 76, 11, 94, 0, 290, 36, 60, 30, 69, 2, 30, 24, 60, 28, 0, 4, 4, 48, 10, 38]


176-element Vector{Int64}:
 2534
  677
 2424
 2162
 1050
  215
  737
  530
 1378
  607
    ⋮
   24
   60
   28
    0
    4
    4
   48
   10
   38

In [32]:
rider_df = 

HTMLElement{:table}:<table class="tablesorter custom-popup">
  <thead>
    <tr>
      <th data-priority="critical" width="10%"></th>
      <th data-placeholder="Search Riders" data-priority="critical" width="23%">
        <b>
          Rider
        </b>
      </th>
      <th data-placeholder="All" data-priority="3" width="23%">
        <b>
          Team
        </b>
      </th>
      <th data-placeholder="All" data-priority="critical" width="20%">
        <b>
          Class
        </b>
      </th>
      <th align="center" data-placeholder="All" data-priority="critical" width="10%">
...
